# A Quick Introduction to Optuna

This Jupyter notebook goes through the basic usage of Optuna.

- Install Optuna
- Write a training algorithm that involves hyperparameters
  - Read train/valid data
  - Define and train model
  - Evaluate model
- Use Optuna to tune the hyperparameters (hyperparameter optimization, HPO)
- Visualize HPO

## Install `optuna`

Optuna can be installed via `pip` or `conda`.

In [1]:
!pip install --quiet optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 7.1 MB/s eta 0:00:00


In [2]:
import optuna

optuna.__version__

'4.5.0'

## Optimize Hyperparameters

### Define a simple scikit-learn model

We start with a simple random forest model to classify flowers in the Iris dataset. We define a function called `objective` that encapsulates the whole training process and outputs the accuracy of the model.

In [3]:
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection


def objective():
    iris = sklearn.datasets.load_iris()  # Prepare the data.

    clf = sklearn.ensemble.RandomForestClassifier(n_estimators=5, max_depth=3)  # Define the model.

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3
    ).mean()  # Train and evaluate the model.


print("Accuracy: {}".format(objective()))

Accuracy: 0.9666666666666667


### Optimize hyperparameters of the model

The hyperparameters of the above algorithm are `n_estimators` and `max_depth` for which we can try different values to see if the model accuracy can be improved. The `objective` function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [4]:
import optuna


def objective(trial):
    iris = sklearn.datasets.load_iris()

    n_estimators = trial.suggest_int("n_estimators", 2, 20)
    max_depth = int(trial.suggest_float("max_depth", 1, 32, log=True))

    clf = sklearn.ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3
    ).mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2025-09-20 07:22:13,331] A new study created in memory with name: no-name-5481adf4-55ce-469f-9800-da2efa588654
[I 2025-09-20 07:22:13,423] Trial 0 finished with value: 0.9466666666666667 and parameters: {'n_estimators': 17, 'max_depth': 1.4110302771783216}. Best is trial 0 with value: 0.9466666666666667.
[I 2025-09-20 07:22:13,452] Trial 1 finished with value: 0.94 and parameters: {'n_estimators': 2, 'max_depth': 7.665787657746182}. Best is trial 0 with value: 0.9466666666666667.
[I 2025-09-20 07:22:13,490] Trial 2 finished with value: 0.9466666666666667 and parameters: {'n_estimators': 5, 'max_depth': 4.1752837736369575}. Best is trial 0 with value: 0.9466666666666667.
[I 2025-09-20 07:22:13,570] Trial 3 finished with value: 0.9466666666666667 and parameters: {'n_estimators': 13, 'max_depth': 11.694206952228047}. Best is trial 0 with value: 0.9466666666666667.
[I 2025-09-20 07:22:13,618] Trial 4 finished with value: 0.9466666666666667 and parameters: {'n_estimators': 6, 'max_depth'

Accuracy: 0.9733333333333333
Best hyperparameters: {'n_estimators': 15, 'max_depth': 3.155010081636098}


It is possible to condition hyperparameters using Python `if` statements. We can for instance include another classifier, a support vector machine, in our HPO and define hyperparameters specific to the random forest model and the support vector machine.

In [5]:
import sklearn.svm


def objective(trial):
    iris = sklearn.datasets.load_iris()

    classifier = trial.suggest_categorical("classifier", ["RandomForest", "SVC"])

    if classifier == "RandomForest":
        n_estimators = trial.suggest_int("n_estimators", 2, 20)
        max_depth = int(trial.suggest_float("max_depth", 1, 32, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth
        )
    else:
        c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)

        clf = sklearn.svm.SVC(C=c, gamma="auto")

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3
    ).mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2025-09-20 07:22:30,029] A new study created in memory with name: no-name-0d4e74bf-f967-4501-af04-bb64445c73fc
[I 2025-09-20 07:22:30,268] Trial 0 finished with value: 0.9466666666666667 and parameters: {'classifier': 'RandomForest', 'n_estimators': 16, 'max_depth': 16.47042865489109}. Best is trial 0 with value: 0.9466666666666667.
[I 2025-09-20 07:22:30,298] Trial 1 finished with value: 0.32 and parameters: {'classifier': 'SVC', 'svc_c': 9.04835335420705e-05}. Best is trial 0 with value: 0.9466666666666667.
[I 2025-09-20 07:22:30,317] Trial 2 finished with value: 0.96 and parameters: {'classifier': 'SVC', 'svc_c': 56.58016300883369}. Best is trial 2 with value: 0.96.
[I 2025-09-20 07:22:30,376] Trial 3 finished with value: 0.96 and parameters: {'classifier': 'RandomForest', 'n_estimators': 9, 'max_depth': 2.522669080813724}. Best is trial 2 with value: 0.96.
[I 2025-09-20 07:22:30,476] Trial 4 finished with value: 0.9533333333333333 and parameters: {'classifier': 'RandomForest', '

Accuracy: 0.9866666666666667
Best hyperparameters: {'classifier': 'SVC', 'svc_c': 3.9653905035679293}


### Plotting the study

Plotting the optimization history of the study.

In [ ]:
optuna.visualization.plot_optimization_history(study)

Plotting the accuracies for each hyperparameter for each trial.

In [ ]:
optuna.visualization.plot_slice(study)

Plotting the accuracy surface for the hyperparameters involved in the random forest model.

In [ ]:
optuna.visualization.plot_contour(study, params=["n_estimators", "max_depth"])